In [2]:
from pycaret.classification import *
import pandas as pd

In [7]:
df = pd.read_csv("features.csv")
df.head()
df = df[df["label"] != "~"]


In [28]:
config = setup( data = df,
                train_size=0.8,
                target="label",
                use_gpu=True,
                fix_imbalance = True,
                feature_selection=True,
                feature_ratio=True,
                session_id=123)

,Description,Value
0,session_id,123
1,Target,label
2,Target Type,Multiclass
3,Label Encoded,"A: 0, N: 1, O: 2"
4,Original Data,"(5791, 8)"
5,Missing Values,False
6,Numeric Features,7
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [29]:
best_model = compare_models(sort = "F1")

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.7573,0.8380,0.6986,0.7566,0.7542,0.5359,0.5382,0.2560
rf,Random Forest Classifier,0.7543,0.8434,0.6923,0.7530,0.7522,0.5314,0.5326,0.6540
et,Extra Trees Classifier,0.7552,0.8383,0.6822,0.7507,0.7505,0.5273,0.5297,0.4800
gbc,Gradient Boosting Classifier,0.7455,0.8446,0.7041,0.7553,0.7465,0.5264,0.5291,5.1670
qda,Quadratic Discriminant Analysis,0.7200,0.8138,0.6852,0.7467,0.7106,0.4745,0.4889,0.0300
ada,Ada Boost Classifier,0.6898,0.7506,0.6624,0.7137,0.6964,0.4411,0.4450,0.5030
lr,Logistic Regression,0.6682,0.7744,0.6482,0.6874,0.6717,0.4002,0.4041,1.7510
dt,Decision Tree Classifier,0.6643,0.7093,0.6125,0.6825,0.6711,0.3878,0.3898,0.0900
knn,K Neighbors Classifier,0.6615,0.7751,0.6375,0.6866,0.6691,0.3928,0.3963,0.0580
ridge,Ridge Classifier,0.6636,0.0000,0.6458,0.6829,0.6634,0.3914,0.3977,0.0240


In [43]:
tuned_model = tune_model(best_model, optimize='F1', n_iter = 100)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7608,0.8336,0.7001,0.7574,0.7551,0.5392,0.5427
1,0.7953,0.8771,0.7344,0.7920,0.7910,0.6070,0.6095
2,0.7689,0.8569,0.7307,0.7723,0.7673,0.5633,0.5654
3,0.7473,0.8371,0.7038,0.7499,0.7458,0.5210,0.5228
4,0.7516,0.8422,0.7093,0.7530,0.7491,0.5290,0.5311
5,0.7279,0.7980,0.6838,0.7250,0.7257,0.4802,0.4808
6,0.7732,0.8421,0.7431,0.7756,0.7737,0.5741,0.5745
7,0.7732,0.8590,0.7172,0.7720,0.7721,0.5685,0.5689
8,0.7775,0.8514,0.6830,0.7713,0.7732,0.5655,0.5667


In [59]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[['min_rate', 'avg_rate', 'max_rate', 'sdnn', 'nn50', 'sdsd', 'rmssd']],
                                                    df[["label"]])


model = LGBMClassifier(
    metric="multi_logloss",
    num_leaves=32
)

model.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               metric='multi_logloss', min_child_samples=20,
               min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
               n_jobs=-1, num_leaves=32, objective=None, random_state=None,
               reg_alpha=0.0, reg_lambda=0.0, silent='warn', subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [60]:
from sklearn.metrics import f1_score

y_pred = model.predict(X_test)

f1_score(y_test, y_pred, average="weighted")

0.7761828291198503

In [ ]:
model.save()